In [1]:
import tensorflow as tf
dense = tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal")

2023-06-26 20:24:43.067366: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
he_avg_init = tf.keras.initializers.VarianceScaling(scale=2., mode="fan_avg",
                                                    distribution="uniform")
dense = tf.keras.layers.Dense(50, activation="sigmoid",
                              kernel_initializer=he_avg_init)


In [3]:
# Leaky ReLU is better than regular ReLU
leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)  # defaults to alpha=0.3
dense = tf.keras.layers.Dense(50, activation=leaky_relu,
                              kernel_initializer="he_normal")


In [4]:
# Batch normalization in keras
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)              4

In [6]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])


In [8]:
# Gradient clipping
optimizer = tf.keras.optimizers.SGD(clipvalue=1.0)
model.compile(optimizer=optimizer)


In [12]:
# extra code – split Fashion MNIST into tasks A and B, then train and save
#              model A to "my_model_A".
import numpy as np
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
pos_class_id = class_names.index("Pullover")
neg_class_id = class_names.index("T-shirt/top")

def split_dataset(X, y):
    y_for_B = (y == pos_class_id) | (y == neg_class_id)
    y_A = y[~y_for_B]
    y_B = (y[y_for_B] == pos_class_id).astype(np.float32)
    old_class_ids = list(set(range(10)) - set([neg_class_id, pos_class_id]))
    for old_class_id, new_class_id in zip(old_class_ids, range(8)):
        y_A[y_A == old_class_id] = new_class_id  # reorder class ids for A
    return ((X[~y_for_B], y_A), (X[y_for_B], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

tf.random.set_seed(42)

model_A = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=(X_valid_A, y_valid_A))
model_A.save("my_model_A")

Epoch 1/20
1376/1376 [==============================] - 2s 1ms/step - loss: 1.2435 - accuracy: 0.6356 - val_loss: 0.7593 - val_accuracy: 0.7659
Epoch 2/20
1376/1376 [==============================] - 2s 1ms/step - loss: 0.6516 - accuracy: 0.7922 - val_loss: 0.5509 - val_accuracy: 0.8235
Epoch 3/20
1376/1376 [==============================] - 2s 1ms/step - loss: 0.5242 - accuracy: 0.8328 - val_loss: 0.4721 - val_accuracy: 0.8403
Epoch 4/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.4592 - accuracy: 0.8513 - val_loss: 0.4221 - val_accuracy: 0.8581
Epoch 5/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.4165 - accuracy: 0.8639 - val_loss: 0.3889 - val_accuracy: 0.8669
Epoch 6/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.3862 - accuracy: 0.8730 - val_loss: 0.3663 - val_accuracy: 0.8744
Epoch 7/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.3642 - accuracy: 0.8784 - val_loss: 0.3489 - val_accuracy:

INFO:tensorflow:Assets written to: my_model_A/assets


INFO:tensorflow:Assets written to: my_model_A/assets


In [13]:
model_A = tf.keras.models.load_model("my_model_A")
model_B_on_A = tf.keras.Sequential(model_A.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))
model_A_clone = tf.keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [14]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])

In [15]:
# Train transfer model
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 29ms/step - loss: 1.8742 - accuracy: 0.4450 - val_loss: 1.2454 - val_accuracy: 0.4847
Epoch 2/4
7/7 [==============================] - 0s 9ms/step - loss: 1.1230 - accuracy: 0.4500 - val_loss: 0.8472 - val_accuracy: 0.4916
Epoch 3/4
7/7 [==============================] - 0s 9ms/step - loss: 0.8492 - accuracy: 0.4350 - val_loss: 0.7699 - val_accuracy: 0.3610
Epoch 4/4
7/7 [==============================] - 0s 10ms/step - loss: 0.7862 - accuracy: 0.3350 - val_loss: 0.7550 - val_accuracy: 0.3927
Epoch 1/16
7/7 [==============================] - 1s 30ms/step - loss: 0.7471 - accuracy: 0.4050 - val_loss: 0.6897 - val_accuracy: 0.5687
Epoch 2/16
7/7 [==============================] - 0s 9ms/step - loss: 0.6806 - accuracy: 0.5600 - val_loss: 0.6281 - val_accuracy: 0.6963
Epoch 3/16
7/7 [==============================] - 0s 9ms/step - loss: 0.6196 - accuracy: 0.7150 - val_loss: 0.5943 - val_accuracy: 0.6884
Epoch 4/16
7/7 [===================

In [16]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 1ms/step - loss: 0.3430 - accuracy: 0.9210


[0.3430161476135254, 0.9210000038146973]

In [17]:
# implementing momentum optimization
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

In [18]:
# Nesterov Accelerated Gradient (faster than momentum)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

In [19]:
# RMS Prop, uses exp decay on most recent gradients
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

In [20]:
# Adam, combination of momentum and RMS prop
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [22]:
# Changing learning rates
def exponential_decay_fn(epoch):
    return 0.01 * 0.1 ** (epoch/20)

In [23]:
# Regularization
layer = tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal",
                              kernel_regularizer=tf.keras.regularizers.l2(0.01))

In [24]:
# Reuse regularization in several layers
from functools import partial

RegularizedDense = partial(tf.keras.layers.Dense,
                           activation="relu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=tf.keras.regularizers.l2(0.01))

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(100),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])

In [32]:
# Dropout
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])

# extra code – compile and train the model
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6943 - accuracy: 0.7439 - val_loss: 0.4625 - val_accuracy: 0.8268
Epoch 2/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.5201 - accuracy: 0.8098 - val_loss: 0.4039 - val_accuracy: 0.8498
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4799 - accuracy: 0.8244 - val_loss: 0.4071 - val_accuracy: 0.8476
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4534 - accuracy: 0.8337 - val_loss: 0.3852 - val_accuracy: 0.8552
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4405 - accuracy: 0.8380 - val_loss: 0.3698 - val_accuracy: 0.8590
Epoch 6/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.4276 - accuracy: 0.8432 - val_loss: 0.3795 - val_accuracy: 0.8552
Epoch 7/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.4194 - accuracy: 0.8455 - val_loss: 0.3581 - val_accuracy:

In [33]:
# Monte Carlo Dropout
import numpy as np

y_probas = np.stack([model(X_test, training=True)
                     for sample in range(100)])
y_proba = y_probas.mean(axis=0)

In [34]:
model.predict(X_test[:1]).round(3)

1/1 [==============================] - 0s 53ms/step


array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.009, 0.   , 0.309, 0.   ,
        0.681]], dtype=float32)

In [35]:
y_proba[0].round(3)

array([0.001, 0.   , 0.   , 0.001, 0.   , 0.027, 0.001, 0.347, 0.002,
       0.622], dtype=float32)

In [36]:
y_std = y_probas.std(axis=0)
y_std[0].round(3)

array([0.002, 0.001, 0.001, 0.003, 0.001, 0.041, 0.003, 0.207, 0.008,
       0.213], dtype=float32)

In [37]:
y_pred = y_proba.argmax(axis=1)
accuracy = (y_pred == y_test).sum() / len(y_test)
accuracy

0.8659

In [38]:
class MCDropout(tf.keras.layers.Dropout):
    def call(self, inputs, training=False):
        return super().call(inputs, training=True)

In [39]:
# Max-norm regularization
dense = tf.keras.layers.Dense(
    100, activation="relu", kernel_initializer="he_normal",
    kernel_constraint=tf.keras.constraints.max_norm(1.))